In [1]:
import pydicom
import os
from pydicom.data import get_testdata_files

def debug_dicom(dcm_root):
    '''
    This function is to see what slice has been lost in a dicom folder
    '''
    dcm_list = os.listdir(dcm_root)
    num_list = []
    for i in range(len(dcm_list)):
        ds = pydicom.dcmread(dcm_root + '/' + dcm_list[i])
#         if int(ds[0x20, 0x13].value) == 137:
#             print (1137)
        num_list.append(int(ds[0x20, 0x13].value))                 # ds[0x20, 0x13]: instance number
    for i in range(1, max(num_list)):
        if i not in num_list:
            print (i)

def debug_distance(dcm_root, th):
    dcm_list = os.listdir(dcm_root)
    num_list = []
    print (dcm_list[0], dcm_list[1])
    ds_0 = pydicom.dcmread(dcm_root + '/EXP0000')  # here should change 
    ds_1 = pydicom.dcmread(dcm_root + '/EXP0001')
    ds_2 = pydicom.dcmread(dcm_root + '/EXP0002')
    if abs(ds_1[0x20, 0x13].value - ds_0[0x20, 0x13].value) >= 1:
        times = float('%.2f' % (ds_1.SliceLocation - ds_0.SliceLocation)) / (ds_1[0x20, 0x13].value - ds_0[0x20, 0x13].value)
    if abs(ds_2[0x20, 0x13].value - ds_0[0x20, 0x13].value) >= 1:
        times = float('%.2f' % (ds_2.SliceLocation - ds_0.SliceLocation)) / (ds_2[0x20, 0x13].value - ds_0[0x20, 0x13].value)
        
    for i in range(len(dcm_list)):
        ds = pydicom.dcmread(dcm_root + '/' + dcm_list[i])
        
        diff = float('%.2f'%float(ds.SliceLocation)) - float('%.2f'%float(ds_0.SliceLocation)) - float('%.2f'%float(times)) * int(ds[0x20, 0x13].value - ds_0[0x20, 0x13].value)
        print (ds.SliceLocation, ds_0.SliceLocation, float(times), int(ds[0x20, 0x13].value - ds_0[0x20, 0x13].value), diff)
#         if diff > th:
#             print (diff, i, dcm_list[i])
def check_slicedistance(dicom_root):
    dcm_list = os.listdir(data_root)
    ds_list = []
    for i in range(len(dcm_list)):
        ds = pydicom.dcmread(data_root + '/' + dcm_list[i])
        ds_list.append(ds.SliceLocation)

    ds_sort = sorted(ds_list)  
    for i in range(0, len(ds_sort) - 2):
        assert ds_sort[i] - ds_sort[i + 1] == ds_sort[i + 1] - ds_sort[i + 2]

def resave_slicelocation(dcm_root, new_root):
    dcm_list = os.listdir(dcm_root)
    num_list = []
    
    ds_0 = pydicom.dcmread(dcm_root + '/EXP0000')  # here should change 
    ds_1 = pydicom.dcmread(dcm_root + '/EXP0001')
    ds_2 = pydicom.dcmread(dcm_root + '/EXP0002')
    if abs(ds_1[0x20, 0x13].value - ds_0[0x20, 0x13].value) == 1:
        times = '%.2f' % (ds_1.SliceLocation - ds_0.SliceLocation)
    if abs(ds_2[0x20, 0x13].value - ds_0[0x20, 0x13].value) == 1:
        times = '%.2f' % (ds_2.SliceLocation - ds_0.SliceLocation)
        
    for i in range(len(dcm_list)):
        ds = pydicom.dcmread(dcm_root + '/' + dcm_list[i])
        ds.SliceLocation = ds_0.SliceLocation + float(times) * (ds[0x20, 0x13].value - ds_0[0x20, 0x13].value)
        ds.SliceLocation = float('%.2f' % ds.SliceLocation)
        ds.save_as(new_root + "/" + dcm_list[i] + '.dcm')
    

In [6]:
import pandas as pd
dcm_root = '/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/MCL/xnat/Xnat0826/MCL/12143271663_12143271663-20151105/12143271663_12143271663-20151105/file0/DICOM'
dcm_list = os.listdir(dcm_root)
slicePos = []
instanceN = []
for i in range(len(dcm_list)):
    ds = pydicom.dcmread(dcm_root + '/' + dcm_list[i], force = True)
    slicePos.append(ds.SliceLocation)
    instanceN.append(ds[0x20, 0x13].value)
    
print (len(instanceN) == max(instanceN) - min(instanceN) + 1)
  
# data = pd.DataFrame()
# data['dicom name'] = dcm_list
# data['slice position'] = slicePos
# data['instance number'] = instanceN
# data.to_csv('/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/MCL/xnat/Xnat0826/MCL/29907914967_29907914967-20130910/29907914967_29907914967-20130910/3-x-1.25-x-1.25/slicePosition2.csv')

True


In [4]:
ds = pydicom.dcmread('/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/MCL/xnat/Xnat052219/MCL/6503414310/6503414310-20141106/new_max/DICOM/1.2.840.113654.2.70.1.324512348527964100210102126208413857752-3-1-1b0bhwo.dcm', force = True)
print (ds)

(0008, 0005) Specific Character Set              CS: 'ISO_IR 100'
(0008, 0008) Image Type                          CS: ['ORIGINAL', 'PRIMARY', 'AXIAL', 'CT_SOM5 SPI']
(0008, 0016) SOP Class UID                       UI: CT Image Storage
(0008, 0018) SOP Instance UID                    UI: 1.2.840.113654.2.70.1.287102932879061839938605094370037197246
(0008, 0020) Study Date                          DA: '20141106'
(0008, 0021) Series Date                         DA: '20141106'
(0008, 0022) Acquisition Date                    DA: '20141106'
(0008, 0023) Content Date                        DA: '20141106'
(0008, 0030) Study Time                          TM: '094745.281000'
(0008, 0031) Series Time                         TM: '095825.390000'
(0008, 0032) Acquisition Time                    TM: '095745.724111'
(0008, 0033) Content Time                        TM: '095745.724111'
(0008, 0050) Accession Number                    SH: '3020338414068298'
(0008, 0060) Modality                       

In [15]:
resave_slicelocation('/share5/gaor2/tmp_file/EXP00002', "/share5/gaor2/tmp_file/new_EXP00002")

In [26]:
data_root = '/media/gaor2/8e7f6ccf-3585-4815-856e-80ce8754c5b5/data/MCL/xnat/Xnat052219/MCL/5319982533/5319982533-20160718/new_max/DICOM'
dcm_list = os.listdir(data_root)
ds_list = []
for i in range(len(dcm_list)):
    ds = pydicom.dcmread(data_root + '/' + dcm_list[i])
    ds_list.append(ds.SliceLocation)
    
ds_sort = sorted(ds_list, reverse = True)
print (ds_sort)
for i in range(0, len(ds_sort) - 2):
        #print (i, ds_sort[i] - ds_sort[i + 1], ds_sort[i + 1] - ds_sort[i + 2])
        assert abs((ds_sort[i] - ds_sort[i + 1]) - (ds_sort[i + 1] - ds_sort[i + 2])) < 0.01
# print (ds_sort[0] - ds_sort[1], ds_sort[1] - ds_sort[2], ds_sort[3] - ds_sort[2], ds_sort[3] - ds_sort[4])
# print (ds_sort)

["333.60", "332.80", "332.00", "331.20", "330.40", "329.60", "328.80", "328.00", "327.20", "326.40", "325.60", "324.80", "324.00", "323.20", "322.40", "321.60", "320.80", "320.00", "319.20", "318.40", "317.60", "316.80", "316.00", "315.20", "314.40", "313.60", "312.80", "312.00", "311.20", "310.40", "309.60", "308.80", "308.00", "307.20", "306.40", "305.60", "304.80", "304.00", "303.20", "302.40", "301.60", "300.80", "300.00", "299.20", "298.40", "297.60", "296.80", "296.00", "295.20", "294.40", "293.60", "292.80", "292.00", "291.20", "290.40", "289.60", "288.80", "288.00", "287.20", "286.40", "285.60", "284.80", "284.00", "283.20", "282.40", "281.60", "280.80", "280.00", "279.20", "278.40", "277.60", "276.80", "276.00", "275.20", "274.40", "273.60", "272.80", "272.00", "271.20", "270.40", "269.60", "268.80", "268.00", "267.20", "266.40", "265.60", "264.80", "264.00", "263.20", "262.40", "261.60", "260.80", "260.00", "259.20", "258.40", "257.60", "256.80", "256.00", "255.20", "254.40",

In [ ]:
import os
#dcm_root = '/share3/gaor2/share5backup/data/MCL/xnat_data/Xnat1115/MCL/7580370015/7580370015_20180124/5-x-AXIAL_-x-AXIAL_/DICOM'

dcm_root = '/share5/gaor2/tmp_file/EXP00002'
dcm_list = os.listdir(dcm_root)

ds_0 = pydicom.dcmread(dcm_root + '/EXP0000')
ds_1 = pydicom.dcmread(dcm_root + '/EXP0001')

times =  (ds_1.SliceLocation - ds_0.SliceLocation)
# print (ds_0.SliceLocation, ds_0[0x20, 0x13].value, ds_1.SliceLocation, ds_1[0x20, 0x13].value, times)
num_list = []
for i in range(1, len(dcm_list)):
 #   if i > 20: break
    print (i)

    ds = pydicom.dcmread(dcm_root + '/' + dcm_list[i])

    print (ds.SliceLocation, float(ds[0x20, 0x13].value), times, ds_0.SliceLocation, dcm_list[i])
#    diff = ds.SliceLocation - (float(ds[0x20, 0x13].value) - 1 ) * float(times) - ds_0.SliceLocation
#    ds.SliceLocation = '%.2f' % ((float(ds[0x20, 0x13].value) - 1 ) * float(times) - ds_0.SliceLocation)
#    print (i,dcm_list[i], diff)

    
#     print (ds.SliceLocation)
#     #print (dcm_root + '/' + dcm_list[i])
#    ds.save_as("/share5/gaor2/tmp_file/new_EXP00002/" + dcm_list[i])
#    num_list.append(int(ds[0x20, 0x13].value))



In [ ]:
len(dcm_list)

In [ ]:
new1 = [(i - num_list[0]) for i in new_list]

In [ ]:
num_list

In [ ]:
for i in range(210):
    if i not in num_list:
        print (i)